## P3 Workshop on 8/10/2018

In [3]:
from Bio.Seq import Seq
from Bio.Alphabet import IUPAC
my_seq = Seq("AAAATGCGTACCTCGGTCGAAGTC", IUPAC.unambiguous_dna)
my_seq

Seq('AAAATGCGTACCTCGGTCGAAGTC', IUPACUnambiguousDNA())

### Exercise 1.1

*Just a quick Python refresher - loop through sequences and print out "Sequence number ... is ... characters long." *


In [4]:
sequences = [Seq("AGCGATGTTACGCATCAGGGCAGTCGCCCTAAAACAAAGTTAGGCCGC", IUPAC.unambiguous_dna),
            Seq("CCGGTTGGTAACGGCGCAGTGGCGGTTTTCAT", IUPAC.unambiguous_dna),
            Seq("CACAGCGGTTTTCAT", IUPAC.unambiguous_dna),
            Seq("AACGGCGCATGGCGGTTTTCAT", IUPAC.unambiguous_dna),
            Seq("AGTGGCGGTTTTCAT", IUPAC.unambiguous_dna)]

In [15]:
x = 1
for seq in sequences:
    print("Sequence number " + str(x) + " is "  + str(len(seq)) + (" characters long."))
    x += 1
    
#or

for seq_num, seq in enumerate(sequences):
    print("Sequence number {} is {} characters long.".format(
    seq_num + 1, len(seq)))

Sequence number 1 is 48 characters long.
Sequence number 2 is 32 characters long.
Sequence number 3 is 15 characters long.
Sequence number 4 is 22 characters long.
Sequence number 5 is 15 characters long.
Sequence number 1 is 48 characters long.
Sequence number 2 is 32 characters long.
Sequence number 3 is 15 characters long.
Sequence number 4 is 22 characters long.
Sequence number 5 is 15 characters long.


### Exercise 1.2

*Let's say you really wanted to concatenate my_prot and my_seq for some reason. How would you do that?*

In [16]:
my_seq = Seq("GATCGATGGGCCTATATAGGATACGAAAATCGC", IUPAC.unambiguous_dna)
my_prot = Seq("EVRNAK", IUPAC.protein)


In [20]:
str(my_seq) + str(my_prot)
#explicitly changing alphabet of my_seq
#from Bio.Alphabet import generic_dna



'GATCGATGGGCCTATATAGGATACGAAAATCGCEVRNAK'

### Exercise 2.1

*Verify for yourself that bac_gene is a valid CDS*

In [22]:
from Bio.Alphabet import generic_dna
bac_gene = Seq("GTGAAAAAGATGCAATCTATCGTACTCGCACTTTCCCTGGTTCTGGTCGCTCCCATGGCA" + 
           "GCACAGGCTGCGGAAATTACGTTAGTCCCGTCAGTAAAATTACAGATAGGCGATCGTGAT" + 
           "AATCGTGGCTATTACTGGGATGGAGGTCACTGGCGCGACCACGGCTGGTGGAAACAACAT" + 
           "TATGAATGGCGAGGCAATCGCTGGCACCTACACGGACCGCCGCCACCGCCGCGCCACCAT" + 
           "AAGAAAGCTCCTCATGATCATCACGGCGGTCATGGTCCAGGCAAACATCACCGCTAA", generic_dna)


`bac_gene` has a whole number of codons:

In [25]:
len(bac_gene) % 3 == 0


True

`bac_gene` starts with a start codon:

In [28]:
bac_gene.startswith("ATG")

False

In [27]:
#bacterial start codon
bac_gene.startswith("GTG")

True

`bac_gene` ends with an end codon:

In [35]:
stop_codons = ["TGA", "TAA", "TAG"]

for codon in stop_codons:
    if bac_gene.endswith(codon):
        print("Yayuh, " + codon)
    

Yayuh, TAA


`bac_gene` can't have internal codons

In [46]:
for i in range(0, len(bac_gene), 3):
    if bac_gene[i:i+3] in stop_codons:
        print("{} found at position {}.".format(codon,i))

TAG found at position 294.


*Look through the translation tables on NCBI and determine which table we should use.  Translate bac_gene with the appropriate arguments.*

In [49]:
bac_table = 11
bac_gene.translate(cds = True, table = 11)

Seq('MKKMQSIVLALSLVLVAPMAAQAAEITLVPSVKLQIGDRDNRGYYWDGGHWRDH...HHR', ExtendedIUPACProtein())

*Alternatively you can just call translate without providing a table or setting cds = True.  What's different?*

In [50]:
bac_gene.translate(table = 11)

Seq('VKKMQSIVLALSLVLVAPMAAQAAEITLVPSVKLQIGDRDNRGYYWDGGHWRDH...HR*', HasStopCodon(ExtendedIUPACProtein(), '*'))

### Exercise 1.3

*There are a lot of other attributes in a BLAST result XML file. See here for a full class diagram. Modify the BLAST output above to include gaps and frame.*

In [57]:
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML

result_handle = NCBIWWW.qblast("blastn", "nt", "8332116")

In [58]:
#write out blast results to XML file
with open('my_blast_results.xml', 'w') as fout:
    fout.write(result_handle.read())
#close result_handle
    result_handle.close()

#reopen local copy of blast results
result_handle = open('my_blast_results.xml')
#parse blast results
blast_records = NCBIXML.parse(result_handle) 

#set E-value cutoff
E_THRESHHOLD = 0.04

#read through and print results
for record in blast_records:
    for alignment in record.alignments:
        for hsp in alignment.hsps:
            if hsp.expect < E_THRESHHOLD:
                print('****Alignment****')
                print('Sequence: {}'.format(alignment.title))
                print('Length: {}'.format(alignment.length))
                print('E value: {}'.format(hsp.expect))
                print('Gaps: {}'.format(hsp.gaps))
                print('Frame: {}'.format(hsp.frame))
                print(hsp.query[0:75] + '...')
                print(hsp.match[0:75] + '...')
                print(hsp.sbjct[0:75] + '...')


****Alignment****
Sequence: gi|1219041180|ref|XM_021875076.1| PREDICTED: Chenopodium quinoa cold-regulated 413 plasma membrane protein 2-like (LOC110697660), mRNA
Length: 1173
E value: 1.21643e-117
Gaps: 4
Frame: (1, 1)
ACAGAAAATGGGGAGAGAAATGAAGTACTTGGCCATGAAAACTGATCAATTGGCCGTGGCTAATATGATCGATTC...
|| ||||||||| |||| | |||| ||  |||| |||| | |||| ||| | |||| ||| ||| ||||| | ||...
ACCGAAAATGGGCAGAGGAGTGAATTATATGGCAATGACACCTGAGCAACTAGCCGCGGCCAATTTGATCAACTC...
****Alignment****
Sequence: gi|1226796956|ref|XM_021992092.1| PREDICTED: Spinacia oleracea cold-regulated 413 plasma membrane protein 2-like (LOC110787470), mRNA
Length: 672
E value: 7.67649e-114
Gaps: 3
Frame: (1, 1)
AAAATGGGGAGAGAAATGAAGTACTTGGCCATGAAAACTGATCAATTGGCCGTGGCTAATATGATCGATTCCGAT...
|||||||| |||  |||| | || ||||| |||||||| || ||||| |||| ||| ||| ||||||||||||||...
AAAATGGGTAGACGAATGGATTATTTGGCGATGAAAACCGAGCAATTAGCCGCGGCCAATTTGATCGATTCCGAT...
****Alignment****
Sequence: gi|731339628|ref|XM_010682658.1| PREDICTED: Beta vulgaris su

*what is this gene?*

In [61]:
mystery = "CTCTCCCAGCTGAGCTATAGCCCCAATGCGCACATAATAAATCGTGTGAACGGGGCGCAT" + \
    "GATATGAGACCCCCGAAACTGTGTCAACGGCTAAATCGATTTCTCGTGTTAAGCGCTGAAAAAGCG" + \
    "GCCAAATCAGCCTGCAAATAACATAATAAGTGGAATGATGTTCACAAATTTGTTGTCACACCGCTG" + \
    "CTGTTATCAAATATAATAAATATCCTCCGGCATAGC"

result_handle = NCBIWWW.qblast("blastn", "nt", mystery)

blast_records = NCBIXML.parse(result_handle) 

#set E-value cutoff
E_THRESHHOLD = 0.04

#read through and print results
for record in blast_records:
    for alignment in record.alignments:
        for hsp in alignment.hsps:
            if hsp.expect < E_THRESHHOLD:
                print('****Alignment****')
                print('Sequence: {}'.format(alignment.title))
                print('Length: {}'.format(alignment.length))
                print('E value: {}'.format(hsp.expect))
                print('Gaps: {}'.format(hsp.gaps))
                print('Frame: {}'.format(hsp.frame))
                print(hsp.query[0:75] + '...')
                print(hsp.match[0:75] + '...')
                print(hsp.sbjct[0:75] + '...')


****Alignment****
Sequence: gi|1148994967|gb|CP019708.1| Yersinia pestis strain 195/P, complete genome
Length: 4655691
E value: 2.48783e-111
Gaps: 0
Frame: (1, 1)
CTCTCCCAGCTGAGCTATAGCCCCAATGCGCACATAATAAATCGTGTGAACGGGGCGCATGATATGAGACCCCCG...
|||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||...
CTCTCCCAGCTGAGCTATAGCCCCAATGCGCACATAATAAATCGTGTGAACGGGGCGCATGATATGAGACCCCCG...
****Alignment****
Sequence: gi|1046912481|gb|CP016273.1| Yersinia pestis strain Cadman chromosome, complete genome
Length: 4602429
E value: 2.48783e-111
Gaps: 0
Frame: (1, -1)
CTCTCCCAGCTGAGCTATAGCCCCAATGCGCACATAATAAATCGTGTGAACGGGGCGCATGATATGAGACCCCCG...
|||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||...
CTCTCCCAGCTGAGCTATAGCCCCAATGCGCACATAATAAATCGTGTGAACGGGGCGCATGATATGAGACCCCCG...
****Alignment****
Sequence: gi|908262105|gb|CP006794.1| Yersinia pestis 1045 sequence
Length: 4502257
E value: 2.48783e-111
Gaps: 0
Frame: (1, -1)
CTCTCCCAGCTGAGCTATAGCCCCAATGCGCACATAATAA

### Exercise 3.1

find avg phred scores

In [67]:
from Bio.SeqRecord import SeqRecord
my_seq_record = SeqRecord(my_seq, id = "WKM1213", description = "Did this work?")
import random
phred_scores = [random.randint(10,60) for _ in range(len(my_seq))]
my_seq_record.letter_annotations['phred_quality'] = phred_scores
my_seq_record.annotations["Average Phred Score"] = sum(my_seq_record.letter_annotations['phred_quality']) / len(my_seq_record)
my_seq_record.annotations["Average Phred Score"]


36.15151515151515

In [68]:
from Bio.SeqUtils import GC
my_seq_record.annotations['GC Content'] = GC(my_seq_record.seq)
my_seq_record.annotations["GC Content"]

45.45454545454545

In [76]:
from Bio import SeqIO
for seq_record in SeqIO.parse("./fastas/fastas/ls_orchid.fasta", "fasta"):
    print(seq_record.id)
    print(repr(seq_record.seq))
    print(len(seq_record))

gi|2765658|emb|Z78533.1|CIZ78533
Seq('CGTAACAAGGTTTCCGTAGGTGAACCTGCGGAAGGATCATTGATGAGACCGTGG...CGC', SingleLetterAlphabet())
740
gi|2765657|emb|Z78532.1|CCZ78532
Seq('CGTAACAAGGTTTCCGTAGGTGAACCTGCGGAAGGATCATTGTTGAGACAACAG...GGC', SingleLetterAlphabet())
753
gi|2765656|emb|Z78531.1|CFZ78531
Seq('CGTAACAAGGTTTCCGTAGGTGAACCTGCGGAAGGATCATTGTTGAGACAGCAG...TAA', SingleLetterAlphabet())
748
gi|2765655|emb|Z78530.1|CMZ78530
Seq('CGTAACAAGGTTTCCGTAGGTGAACCTGCGGAAGGATCATTGTTGAAACAACAT...CAT', SingleLetterAlphabet())
744
gi|2765654|emb|Z78529.1|CLZ78529
Seq('ACGGCGAGCTGCCGAAGGACATTGTTGAGACAGCAGAATATACGATTGAGTGAA...AAA', SingleLetterAlphabet())
733
gi|2765652|emb|Z78527.1|CYZ78527
Seq('CGTAACAAGGTTTCCGTAGGTGAACCTGCGGAAGGATCATTGTTGAGACAGTAG...CCC', SingleLetterAlphabet())
718
gi|2765651|emb|Z78526.1|CGZ78526
Seq('CGTAACAAGGTTTCCGTAGGTGAACCTGCGGAAGGATCATTGTTGAGACAGTAG...TGT', SingleLetterAlphabet())
730
gi|2765650|emb|Z78525.1|CAZ78525
Seq('TGTTGAGATAGCAGAATATACATCGAGTGAATCCGGAGGACCTGTGGTTATTCG...GC

In [82]:
for seq_record in SeqIO.parse("./fastas/fastas/ls_orchid.gbk", 'genbank'):
    print(seq_record.id)
    print(repr(seq_record.seq))
    print(len(seq_record))

Z78533.1
Seq('CGTAACAAGGTTTCCGTAGGTGAACCTGCGGAAGGATCATTGATGAGACCGTGG...CGC', IUPACAmbiguousDNA())
740
Z78532.1
Seq('CGTAACAAGGTTTCCGTAGGTGAACCTGCGGAAGGATCATTGTTGAGACAACAG...GGC', IUPACAmbiguousDNA())
753
Z78531.1
Seq('CGTAACAAGGTTTCCGTAGGTGAACCTGCGGAAGGATCATTGTTGAGACAGCAG...TAA', IUPACAmbiguousDNA())
748
Z78530.1
Seq('CGTAACAAGGTTTCCGTAGGTGAACCTGCGGAAGGATCATTGTTGAAACAACAT...CAT', IUPACAmbiguousDNA())
744
Z78529.1
Seq('ACGGCGAGCTGCCGAAGGACATTGTTGAGACAGCAGAATATACGATTGAGTGAA...AAA', IUPACAmbiguousDNA())
733
Z78527.1
Seq('CGTAACAAGGTTTCCGTAGGTGAACCTGCGGAAGGATCATTGTTGAGACAGTAG...CCC', IUPACAmbiguousDNA())
718
Z78526.1
Seq('CGTAACAAGGTTTCCGTAGGTGAACCTGCGGAAGGATCATTGTTGAGACAGTAG...TGT', IUPACAmbiguousDNA())
730
Z78525.1
Seq('TGTTGAGATAGCAGAATATACATCGAGTGAATCCGGAGGACCTGTGGTTATTCG...GCA', IUPACAmbiguousDNA())
704
Z78524.1
Seq('CGTAACAAGGTTTCCGTAGGTGAACCTGCGGAAGGATCATTGTTGAGATAGTAG...AGC', IUPACAmbiguousDNA())
740
Z78523.1
Seq('CGTAACCAGGTTTCCGTAGGTGAACCTGCGGCAGGATCATTGTTGAGACAGCAG...AAG', IUPAC

In [85]:
identifiers  = []
for seq_record in SeqIO.parse("./fastas/fastas/ls_orchid.gbk", 'genbank'):
    identifiers.append(seq_record.id)
print(identifiers)

['Z78533.1', 'Z78532.1', 'Z78531.1', 'Z78530.1', 'Z78529.1', 'Z78527.1', 'Z78526.1', 'Z78525.1', 'Z78524.1', 'Z78523.1', 'Z78522.1', 'Z78521.1', 'Z78520.1', 'Z78519.1', 'Z78518.1', 'Z78517.1', 'Z78516.1', 'Z78515.1', 'Z78514.1', 'Z78513.1', 'Z78512.1', 'Z78511.1', 'Z78510.1', 'Z78509.1', 'Z78508.1', 'Z78507.1', 'Z78506.1', 'Z78505.1', 'Z78504.1', 'Z78503.1', 'Z78502.1', 'Z78501.1', 'Z78500.1', 'Z78499.1', 'Z78498.1', 'Z78497.1', 'Z78496.1', 'Z78495.1', 'Z78494.1', 'Z78493.1', 'Z78492.1', 'Z78491.1', 'Z78490.1', 'Z78489.1', 'Z78488.1', 'Z78487.1', 'Z78486.1', 'Z78485.1', 'Z78484.1', 'Z78483.1', 'Z78482.1', 'Z78481.1', 'Z78480.1', 'Z78479.1', 'Z78478.1', 'Z78477.1', 'Z78476.1', 'Z78475.1', 'Z78474.1', 'Z78473.1', 'Z78472.1', 'Z78471.1', 'Z78470.1', 'Z78469.1', 'Z78468.1', 'Z78467.1', 'Z78466.1', 'Z78465.1', 'Z78464.1', 'Z78463.1', 'Z78462.1', 'Z78461.1', 'Z78460.1', 'Z78459.1', 'Z78458.1', 'Z78457.1', 'Z78456.1', 'Z78455.1', 'Z78454.1', 'Z78453.1', 'Z78452.1', 'Z78451.1', 'Z78450.1', 'Z7

In [87]:
#or
identifiers = [seq_record.id for seq_record in SeqIO.parse("./fastas/fastas/ls_orchid.gbk", 'genbank')]
print(identifiers)

['Z78533.1', 'Z78532.1', 'Z78531.1', 'Z78530.1', 'Z78529.1', 'Z78527.1', 'Z78526.1', 'Z78525.1', 'Z78524.1', 'Z78523.1', 'Z78522.1', 'Z78521.1', 'Z78520.1', 'Z78519.1', 'Z78518.1', 'Z78517.1', 'Z78516.1', 'Z78515.1', 'Z78514.1', 'Z78513.1', 'Z78512.1', 'Z78511.1', 'Z78510.1', 'Z78509.1', 'Z78508.1', 'Z78507.1', 'Z78506.1', 'Z78505.1', 'Z78504.1', 'Z78503.1', 'Z78502.1', 'Z78501.1', 'Z78500.1', 'Z78499.1', 'Z78498.1', 'Z78497.1', 'Z78496.1', 'Z78495.1', 'Z78494.1', 'Z78493.1', 'Z78492.1', 'Z78491.1', 'Z78490.1', 'Z78489.1', 'Z78488.1', 'Z78487.1', 'Z78486.1', 'Z78485.1', 'Z78484.1', 'Z78483.1', 'Z78482.1', 'Z78481.1', 'Z78480.1', 'Z78479.1', 'Z78478.1', 'Z78477.1', 'Z78476.1', 'Z78475.1', 'Z78474.1', 'Z78473.1', 'Z78472.1', 'Z78471.1', 'Z78470.1', 'Z78469.1', 'Z78468.1', 'Z78467.1', 'Z78466.1', 'Z78465.1', 'Z78464.1', 'Z78463.1', 'Z78462.1', 'Z78461.1', 'Z78460.1', 'Z78459.1', 'Z78458.1', 'Z78457.1', 'Z78456.1', 'Z78455.1', 'Z78454.1', 'Z78453.1', 'Z78452.1', 'Z78451.1', 'Z78450.1', 'Z7